In [ ]:
from mitigate_disparity import mitigate_disparity

est = mitigate_disparity(
    dataset='data/mimic/development_dataset.train.csv',
    protected_features=[
        'ethnicity',
        'gender',
        'insurance'
    ]
)

categorical features: ['insurance', 'ethnicity']
numeric features: ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'prev_adm']
dataset: data/mimic/development_dataset.train.csv
protected_features: ['ethnicity', 'gender', 'insurance']
running 32 processes
groups ['ethnicity', 'gender', 'insurance']
number of variables: 121
number of objectives: 2
checkpoint file: checkpoint.8f632eeb-9d65-48fb-85c3-50e72ae14c2e.pkl
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       32 |      2 |             - |             -
     2 |       64 |      2 |  0.0883720930 |         ideal
     3 |       96 |      3 |  0.2170635728 |             f
     4 |      128 |      3 |  0.0046296296 |         ideal
     5 |      160 |      4 |  0.0168602137 |         ideal
     6 |      192 |      6 |  0.0312500000 |         ideal
     7 |      224 |      7 |  0.0621750655 |             f
     8 |      256 |      6 |  0.0088495575 |         nadir
     9 |      288 

# visualize final front

In [ ]:
import pickle
with open('estimator.pkl','rb') as f:
    est = pickle.load(f)
est.plot().show()

# save video of optimization

In [ ]:
from pyrecorder.recorder import Recorder
from pyrecorder.writers.video import Video
# from pyrecorder.writers.streamer import Streamer
from pymoo.visualization.scatter import Scatter
import matplotlib.pyplot as plt
with open('estimator.pkl','rb') as f:
    est = pickle.load(f)
# use the video writer as a resource
filename = "xgb_nsga3_mlp.mp4"
with Recorder(Video(filename, fps=2)) as rec:
    # for each algorithm object in the history
    for entry in est.res_.history:
        sc = Scatter(title=("Gen %s" % entry.n_gen),
                     labels=['Overall False Positive Rate (FPR)', 'Subgroup False Negative Rate (FNR) Violation']
                    )
        sc.add(entry.pop.get("F"))
#         sc.add(entry.pop.get("F"), plot_type="line", color="black", alpha=0.7)
        sc.do()
#         plt.xlim([0.02, 0.1])
#         plt.ylim([0.29, 0.39])
        # finally record the current visualization to the video
        rec.record()
    

# check test set performance

In [ ]:
from utils import make_measure_dataset
import pandas as pd

import pickle
with open('estimator.pkl','rb') as f:
    est = pickle.load(f)
    
df_test = pd.read_csv('data/mimic/development_dataset.test.csv')
X_test = df_test.drop(columns='binary outcome')
y_test = df_test['binary outcome']
make_measure_dataset(est, 'fomo', X_test, y_test)

In [ ]:
from measure_disparity import measure_disparity
df_fairness = measure_disparity('fomo_model_mimic4_admission.csv')